# Lecture 10
## Convolutional Neural Network

### Concolutional Layer

In [ ]:
import torch

In [ ]:
in_channels, out_channels = 5,10
width, height = 100, 100
kernel_size = 3
batch_size = 1

In [ ]:
input = torch.randn(batch_size,in_channels,width,height)
conv_layer = torch.nn.Conv2d(in_channels,out_channels,kernel_size=kernel_size)
output = conv_layer(input)

print(input.shape)
print(output.shape)
print(conv_layer.weight.shape)

torch.Size([1, 5, 100, 100])
torch.Size([1, 10, 98, 98])
torch.Size([10, 5, 3, 3])


### Padding & Strding

In [ ]:
input = [3,4,5,6,7,
         2,4,6,8,2,
         1,6,7,8,4,
         9,7,4,6,2,
         3,7,5,4,1]
input = torch.Tensor(input).view(1,1,5,5)
conv_layer = torch.nn.Conv2d(1,1,kernel_size=3,stride=2,bias=False)
kernel = torch.Tensor([1,2,3,4,5,6,7,8,9]).view(1,1,3,3)
conv_layer.weight.data = kernel.data
output = conv_layer(input)
print(output)

tensor([[[[208., 263.],
          [251., 169.]]]], grad_fn=<ConvolutionBackward0>)


In [ ]:
input = [3,4,6,5,
         2,4,6,8,
         1,6,7,8,
         9,7,4,6
         ]
input = torch.Tensor(input).view(1,1,4,4)
maxpooling_layer = torch.nn.MaxPool2d(kernel_size=2)
output = maxpooling_layer(input)
print(output)

tensor([[[[4., 8.],
          [9., 8.]]]])


## Convolutional Neural Network Implementations

In [ ]:
class NNet(torch.nn.Module):
  def __init__(self):
    super(NNet,self).__init__()
    self.conv1 = torch.nn.Conv2d(1,10,kernel_size=5)
    self.conv2 = torch.nn.Conv2d(10,20,kernel_size=5)
    self.l1 = torch.nn.Linear(320,10)
    self.activation = torch.nn.ReLU()
    self.pooling = torch.nn.MaxPool2d(2)

  def forward(self,x):
    batch_size = x.size(0)
    y_pred = self.pooling(self.activation(self.conv1(x)))
    y_pred = self.pooling(self.activation(self.conv2(y_pred)))
    y_pred = y_pred.view(batch_size,-1)
    y_pred = self.l1(y_pred)
    return y_pred

In [ ]:
class NNet2(torch.nn.Module):
  def __init__(self):
    super(NNet2,self).__init__()
    self.conv1 = torch.nn.Conv2d(1,10,kernel_size=5)
    self.conv2 = torch.nn.Conv2d(10,20,kernel_size=5)
    self.l1 = torch.nn.Linear(320,10)
    self.activation = torch.nn.ReLU()
    self.pooling = torch.nn.MaxPool2d(2)

  def forward(self,x):
    batch_size = x.size(0)
    y_pred = self.pooling(self.activation(self.conv1(x)))
    y_pred = self.pooling(self.activation(self.conv2(y_pred)))
    y_pred = y_pred.view(batch_size,-1)
    y_pred = self.l1(y_pred)
    return y_pred

In [ ]:
model = NNet()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
print(device)

cuda:0


In [ ]:
model.to(device)

NNet(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (l1): Linear(in_features=320, out_features=10, bias=True)
  (activation): ReLU()
  (pooling): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)

In [ ]:
import numpy as np
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

In [ ]:
batch_size = 64
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,))
])

In [ ]:
train_dataset = datasets.MNIST(
    root="../",
    train=True,
    download=True,
    transform=transform
)

train_loader = DataLoader(
    train_dataset,
    shuffle=True,
    batch_size=batch_size
)

test_dataset = datasets.MNIST(
    root="../",
    train=False,
    download=True,
    transform=transform
)

test_loader = DataLoader(
    test_dataset,
    shuffle=False,
    batch_size=batch_size
)

100%|██████████| 9912422/9912422 [00:00<00:00, 171248501.71it/s]

Extracting ../MNIST/raw/train-images-idx3-ubyte.gz to ../MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 96753749.06it/s]


Extracting ../MNIST/raw/train-labels-idx1-ubyte.gz to ../MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 92381867.93it/s]

Extracting ../MNIST/raw/t10k-images-idx3-ubyte.gz to ../MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 22760488.37it/s]


Extracting ../MNIST/raw/t10k-labels-idx1-ubyte.gz to ../MNIST/raw



In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=0.01)

In [ ]:
def train(epoch):
  running_loss = 0.0
  for batch_idx,data in enumerate(train_loader,0):
    inputs,target = data
    inputs, target = inputs.to(device), target.to(device)
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs,target)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
    if batch_idx%100==299:
      print(epoch,batch_idx,running_loss/300)
      running_loss=0.0

In [ ]:
def test():
  correct = 0
  total = 0
  with torch.no_grad():
    for data in train_loader:
      images,labels = data
      images,labels = images.to(device), labels.to(device)
      outputs = model(images)
      _,pred = torch.max(outputs.data,dim=1)
      total += labels.size(0)
      correct += (pred==labels).sum().item()
  print("Accuracy Rate:",(100*correct/total),"%")

In [ ]:
for epoch in range(10):
  train(epoch)
  test()

Accuracy Rate: 91.34333333333333 %
Accuracy Rate: 96.78166666666667 %
Accuracy Rate: 97.09833333333333 %
Accuracy Rate: 97.745 %
Accuracy Rate: 97.88833333333334 %
Accuracy Rate: 98.25166666666667 %
Accuracy Rate: 98.40333333333334 %
Accuracy Rate: 98.425 %
Accuracy Rate: 97.53666666666666 %
Accuracy Rate: 98.59666666666666 %
